# Classifying movie reviews
**A binary classification example**

Two-class classification, or binary classification, may be the most widely applied kind of machine-learning problem.
In this example, movie reviews will be classified as positive or negative, based on the text content of the reviews.

## The IMDB dataset
You’ll work with the [IMDB dataset](https://www.imdb.com/): a set of 50,000 highly polarized reviews from the Internet Movie Database. They’re split into 25,000 reviews for training and 25,000 reviews for testing, each set consisting of 50% negative and 50% positive reviews.